<a href="https://colab.research.google.com/github/UTK8005/Agentic-Patterns/blob/main/ROUTING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install langchain langchain-community langchain-openai langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.2.12
    Uninstalling langchain-core-1.2.12:
      Successfully uninstalled langchain-core-1.2.12
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough,RunnableBranch
from google.colab import userdata

In [6]:
llm = ChatOpenAI(
    api_key=userdata.get('OPEN_ROUTER_KEY'),
    base_url="https://openrouter.ai/api/v1",
    model="openai/gpt-oss-20b:free",
)

In [7]:
def booking_handler(request:str) -> str:
  '''Handles booking request'''
  print("\n-----------Initialising Booking Handler ------------------\n")
  return f" Simulated Booking request: {request}"

def info_handler(request: str) -> str:
  """Simulates the Info Agent handling a request."""
  print("\n--- DELEGATING TO INFO HANDLER ---")
  return f"Info Handler processed request: '{request}'. Result:Simulated information retrieval."
def unclear_handler(request: str) -> str:
  """Handles requests that couldn't be delegated."""
  print("\n--- HANDLING UNCLEAR REQUEST ---")
  return f"Coordinator could not delegate request: '{request}'.Please clarify."


In [13]:
# Co-ordinator router prompt

co_ordinator_prompt=ChatPromptTemplate.from_template(
     """Analyze the user's request and determine which
specialist handler should process it.
- If the request is related to booking flights or hotels,
output 'booker'.
- For all other general information questions, output 'info'.
- If the request is unclear or doesn't fit either category,
output 'unclear'.
ONLY output one word: 'booker', 'info', or 'unclear'.
{request}
"""
)

if llm:
  coordinator_router_chain = co_ordinator_prompt | llm |StrOutputParser()


In [14]:
# Use RunnableBranch to route based on the router chain's output.
# Define the branches for the RunnableBranch
branches = {
    "booker":RunnablePassthrough.assign(output = lambda x: booking_handler['request']['request']),
    "info":RunnablePassthrough.assign(output = lambda x: info_handler['request']['request']),
    "unclear":RunnablePassthrough.assign(output = lambda x: unclear_handler['request']['request'])
}

In [17]:
delegation_branch =RunnableBranch(
    (lambda x:x['decision'].strip() == "booking",branches["booker"]),
    (lambda x:x['decision'].strip() =="info",branches["info"]),
    branches["unclear"] # one should be default for failsafe
)

In [18]:
# Combine the router chain and the delegation branch into a single runnable
# The router chain's output ('decision') is passed along with the original input ('request')
# to the delegation_branch.
coordinator_agent = {
"decision": coordinator_router_chain,
"request": RunnablePassthrough()
} | delegation_branch | (lambda x: x['output'])

In [ ]:
#-------Example zusage------------------------------
print("--- Running with a booking request ---")
request_a = "Book me a flight to London."
result_a = coordinator_agent.invoke({"request": request_a})
print(f"Final Result A: {result_a}")